# Subdomains

Sometimes our problem may involve multiple subdomains.


In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *   # Opencascade for geometry modeling

In [2]:
# We first create a rectange of size 2x2

outer = Rectangle(2, 2).Face()
outer.edges.name="outer"
outer.edges.Max(X).name = "right"
outer.edges.Min(X).name = "left"
outer.edges.Min(Y).name = "bottom"
outer.edges.Max(Y).name = "top"

In [3]:
# We create a smaller rectangle inside the large one. 

inner = MoveTo(1, 0.9).Rectangle(0.3, 0.5).Face()
inner.edges.name="interface"
inner.maxh = 0.05

# We also create a circle inside the large one
circle = MoveTo(0.5, 0.5).Circle(0.25).Face()
circle.edges.name="circle-interface"
circle.maxh = 0.02

# subtraction!

outer = outer - inner - circle 

inner.faces.name="inner"
inner.faces.col = (1, 0, 0) # color
circle.faces.name = "circle"
circle.faces.col = (0,0,1)
outer.faces.name="outer"

geo = Glue([inner, circle, outer])
Draw(geo);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [4]:
# This maxh is a global one.

mesh = Mesh(OCCGeometry(geo, dim=2).GenerateMesh(maxh=0.2)) 
Draw(mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

# Other domain shapes

We can create other domain shapes easily using the same syntax. 

For instance, if we want the rectangle and circle regions to be a cavity, we only have to use 

```
geo = Glue([outer])
Draw(geo);
```

to create the geometry on the outer medium only.

In [5]:
cavity_geo = Glue([outer])
Draw(cavity_geo);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [6]:
cavity_mesh = Mesh(OCCGeometry(cavity_geo, dim=2).GenerateMesh(maxh=0.05)) 
Draw(cavity_mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

Let's construct an L-shape. 

In [7]:
L1 = Rectangle(1, 2).Face()
L1.edges.name="outer"


L2 = MoveTo(1, 0).Rectangle(1, 1).Face()
L2.edges.name="interface"

L1 = L2 + L1

Draw(L1);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

In [8]:
L_mesh = Mesh(OCCGeometry(L1, dim=2).GenerateMesh(maxh=0.2)) 
Draw(L_mesh);

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

In [9]:
L_mesh.GetBoundaries()

('interface', 'interface', 'outer', 'outer', 'outer', 'interface')

# Solve Poisson Equation on L-shape

We simply copy all of the previous codes here (the mesh has been changed).

In [10]:
f = sin(x)
fes = H1(L_mesh, order=2, dirichlet=[1,2,3,4,5,6])

In [11]:
g = sin(x)
u_D = GridFunction(fes)
u_D.Set(g, BND) # BND means interpolation on (near) boundary only
Draw(u_D)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [12]:
u = fes.TrialFunction()  # symbolic object
v = fes.TestFunction()   # symbolic object
sol = GridFunction(fes)  # solution

a = BilinearForm(grad(u)*grad(v)*dx)
L = LinearForm(f * v * dx) 

a.Assemble()
L.Assemble()

r = L.vec - a.mat * u_D.vec

sol.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * r + u_D.vec.data

Draw(sol)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.24…

BaseWebGuiScene

In [13]:
true_sol = sin(x)
Integrate( (sol - true_sol) **2 , L_mesh)

1.715254106191511e-09